![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FFramework+Workflows%2FR&file=R+on+Vertex+AI+Pipelines.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/R/R%20on%20Vertex%20AI%20Pipelines.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/R/R%20on%20Vertex%20AI%20Pipelines.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FFramework%2520Workflows%2FR%2FR%2520on%2520Vertex%2520AI%2520Pipelines.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/R/R%20on%20Vertex%20AI%20Pipelines.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery logo">
      <br>Open in<br>BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Framework%20Workflows/R/R%20on%20Vertex%20AI%20Pipelines.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# R on Vertex AI Pipelines

Run an R script in a Vertex AI Pipeline use KFP components.

In [40]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [41]:
REGION = 'us-central1'
SERIES = 'frameworks'
EXPERIMENT = 'r-pipeline'

# gcs bucket
GCS_BUCKET = PROJECT_ID

In [42]:
import os, typing
from google.cloud import aiplatform
import kfp

In [43]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

In [44]:
DIR = f"temp/{SERIES}-{EXPERIMENT}"

In [45]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:
- make a local folder for temporary storage

In [46]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

In [47]:
kfp.local.init(
    runner = kfp.local.DockerRunner(),
    pipeline_root = DIR
)

---
# R CODE

In [1]:
r_file = '../../08 - R/code/train.R'

In [2]:
with open(r_file, 'r') as file:
    r_script = file.read()

In [3]:
print(r_script)

# library import
library(bigrquery)
library(dplyr)

# inputs
args <- commandArgs(trailingOnly = TRUE)
bq_project <- args[1]
bq_dataset <- args[2]
bq_table <- args[3]
var_target <- args[4]
var_omit <- args[5]

# data source
get_data <- function(s){
    
    # query for table
    query <- sprintf('
        SELECT * EXCEPT(%s)
        FROM `%s.%s.%s`
        WHERE splits = "%s"
    ', var_omit, bq_project, bq_dataset, bq_table, s)
    
    # connect to table
    table <- bq_project_query(bq_project, query)
    
    # load table to dataframe
    return(bq_table_download(table, n_max = Inf))

}
train <- get_data("TRAIN")
test <- get_data("TEST")

# logistic regression model
model_exp = paste0(var_target, "~ .")

model <- glm(
    as.formula(model_exp),
    data = train,
    family = binomial)

# predictions for evaluation
preds <- predict(model, test, type = "response")

# evaluate
actual <- test[, var_target]
names(actual) <- 'actual'
pred <- tibble(round(preds))
names(pred) <- 'pred'
resu

In [51]:
test_script = r_script[0:208] + r_script[-76:]
print(test_script)

# library import
library(bigrquery)
library(dplyr)

# inputs
args <- commandArgs(trailingOnly = TRUE)
bq_project <- args[1]
bq_dataset <- args[2]
bq_table <- args[3]
var_target <- args[4]
var_omit <- args[5]

# output
writeLines(paste('project:', bq_project, 'dataset:', bq_dataset))



---
# R Container

In [ ]:
#!gcloud container images list --repository="us-docker.pkg.dev/vertex-ai/training"

In [60]:
#!gcloud container images list --repository="gcr.io/deeplearning-platform-release"

---
# PIPELINE

In [54]:
@kfp.dsl.component(
    base_image = 'python:3.11'
)
def r_job(
    r_args: list,
    r_libs: list,
    r_script: str,
    output_dir: str
) -> kfp.dsl.Artifact:
    
    import base64
    
    # libs
    r_libs_str = ', '.join([repr(r) for r in r_libs])
    install_command = f"""R -e "install.packages(c({r_libs_str}), repos=\'http://cran.us.r-project.org\')" &&"""
    
    # args
    #script_args = ' '.join([repr(arg) for arg in r_args])
    script_args = ' '.join(r_args)
    
    # script
    r_script_bytes = r_script.encode('utf-8')
    encoded_r_script = base64.b64encode(r_script_bytes).decode('utf-8')
    
    # output artifact
    job = kfp.dsl.Artifact(
        metadata = dict(
            install_command = install_command,
            script_args = script_args,
            encoded_r_script = encoded_r_script,
            output_dir = output_dir
        )
    )
    
    return job


@kfp.dsl.container_component
def r_container(
    r_job: kfp.dsl.Input[kfp.dsl.Artifact]
):

    return kfp.dsl.ContainerSpec(
        #image = 'r-base:latest',
        image = 'gcr.io/deeplearning-platform-release/r-cpu.4-4',
        command = ['sh', '-c'],
        args = [
            """
            apt-get update && apt-get install -y libcurl4-openssl-dev libssl-dev libxml2-dev && 
            export AIP_MODEL_DIR={output_dir} &&
            {install_command} 
            echo "{encoded_r_script}" | base64 -d > r_script.R &&
            Rscript r_script.R {script_args}
            """.format(
                output_dir = r_job.metadata['output_dir'],
                install_command = r_job.metadata['install_command'],
                encoded_r_script = r_job.metadata['encoded_r_script'],
                script_args = r_job.metadata['script_args']
            )
        ]
    )


@kfp.dsl.pipeline(
    name = f'{SERIES}-{EXPERIMENT}',
)
def r_pipeline(
    r_libs: list,
    r_args: list,
    r_script: str,
    output_dir: str
) -> kfp.dsl.Artifact:
    
    r_job_op = r_job(
        r_args = r_args,
        r_libs = r_libs,
        r_script = r_script,
        output_dir = output_dir
    )
    
    r_run_op = r_container(r_job = r_job_op.output)
    
    return r_job_op.output

In [55]:
kfp.compiler.Compiler().compile(
    pipeline_func = r_pipeline,
    package_path = f'{DIR}/{SERIES}-{EXPERIMENT}.yaml'
)

In [56]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}.yaml",
    parameter_values = dict(
        r_libs = ['dplyr', 'bigrquery'],
        r_args = [PROJECT_ID, 'r', 'bigquery-data', 'Class', 'transaction_id,splits'],
        r_script = r_script,
        output_dir = f"gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}"
    ),
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = None # True (enabled), False (disable), None (defer to component level caching) 
)

In [57]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/frameworks-r-pipeline-20250326184359?project=1026793852137


In [58]:
pipeline_job.wait()

PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob run completed. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/frameworks-r-pipeline-20250326184359


In [59]:
aiplatform.get_pipeline_df(pipeline = f'{SERIES}-{EXPERIMENT}')

,pipeline_name,run_name,param.vmlmd_lineage_integration,param.input:output_dir,param.vertex-ai-pipelines-artifact-argument-binding,param.input:r_libs,param.input:r_script,param.input:r_args,metric.output_dir,metric.encoded_r_script,metric.install_command,metric.script_args
0,frameworks-r-pipeline,frameworks-r-pipeline-20250326184359,{'pipeline_run_component': {'location_id': 'us...,gs://statmike-mlops-349915/frameworks/r-pipeline,{'output:Output': ['projects/1026793852137/loc...,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,"[statmike-mlops-349915, r, bigquery-data, Clas...",gs://statmike-mlops-349915/frameworks/r-pipeline,IyBsaWJyYXJ5IGltcG9ydApsaWJyYXJ5KGJpZ3JxdWVyeS...,"R -e ""install.packages(c('dplyr', 'bigrquery')...",statmike-mlops-349915 r bigquery-data Class tr...
1,frameworks-r-pipeline,frameworks-r-pipeline-20250326182816,{'pipeline_run_component': {'location_id': 'us...,gs://statmike-mlops-349915/frameworks/r-pipeline,{'output:Output': ['projects/1026793852137/loc...,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,"[statmike-mlops-349915, r, bigquery-data, Clas...",gs://statmike-mlops-349915/frameworks/r-pipeline,IyBsaWJyYXJ5IGltcG9ydApsaWJyYXJ5KGJpZ3JxdWVyeS...,"R -e ""install.packages(c('dplyr', 'bigrquery')...",statmike-mlops-349915 r bigquery-data Class tr...
2,frameworks-r-pipeline,frameworks-r-pipeline-20250326174913,{'pipeline_run_component': {'parent_task_names...,gs://statmike-mlops-349915/frameworks/r-pipeline,{'output:Output': ['projects/1026793852137/loc...,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,"[statmike-mlops-349915, r, bigquery-data, Clas...",gs://statmike-mlops-349915/frameworks/r-pipeline,IyBsaWJyYXJ5IGltcG9ydApsaWJyYXJ5KGJpZ3JxdWVyeS...,"R -e ""install.packages(c('dplyr', 'bigrquery')...",statmike-mlops-349915 r bigquery-data Class tr...
3,frameworks-r-pipeline,frameworks-r-pipeline-20250326171343,{'pipeline_run_component': {'location_id': 'us...,gs://statmike-mlops-349915/frameworks/r-pipeline,{'output:Output': ['projects/1026793852137/loc...,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,"[statmike-mlops-349915, r, bigquery-data, Clas...",gs://statmike-mlops-349915/frameworks/r-pipeline,IyBsaWJyYXJ5IGltcG9ydApsaWJyYXJ5KGJpZ3JxdWVyeS...,"R -e ""install.packages(c('dplyr', 'bigrquery')...",'statmike-mlops-349915' 'r' 'bigquery-data' 'C...


---
# TODO

- ~~set environment variable for AIP_MODEL_DIR~~
- make installs and args optional
- set outputs and pass back to component
- create output artifacts - can have the script > output.txt or something like that potentially
- writeup
- move to frameworks/R
- make dataprep standard from frameworks
- move other R example to frameworks/R and standardize dataprep to frameworks approach of shared table


---
# Development Work

In [79]:
bq_project = PROJECT_ID
bq_dataset = SERIES
bq_table = EXPERIMENT
var_target = 'Class'
var_omit = 'transaction_id'

In [112]:
@kfp.dsl.component(
    base_image = 'python:3.11'
)
def r_job(
    r_args: list,
    r_libs: list,
    r_script: str
) -> kfp.dsl.Artifact:
    
    import base64
    
    r_libs_str = ', '.join([repr(r) for r in r_libs])
    install_command = f"""R -e "install.packages(c({r_libs_str}), repos=\'http://cran.us.r-project.org\')" &&"""
    script_args = ' '.join([repr(arg) for arg in r_args])
    #escaped_r_script = r_script.replace("'", "'\\'").replace("(", "\\(").replace(")", "\\)")
    #escaped_r_script = r_script.replace("'", "'\\'")
    
    r_script_bytes = r_script.encode('utf-8')
    encoded_r_script = base64.b64encode(r_script_bytes).decode('utf-8')
    
    job = kfp.dsl.Artifact(
        metadata = dict(
            install_command = install_command,
            script_args = script_args,
            encoded_r_script = encoded_r_script
        )
    )
    
    return job

In [113]:
test_r_job = r_job(r_args = ['a', 'b'], r_libs = ['c', 'd'], r_script = 'my script here')

14:34:14.928 - INFO - Executing task 'r-job'
14:34:14.934 - INFO - Streamed logs:

    Found image 'python:3.11'

    [KFP Executor 2025-03-26 14:34:21,055 INFO]: Looking for component `r_job` in --component_module_path `/tmp/tmp.Y35uX5nk4z/ephemeral_component.py`
    [KFP Executor 2025-03-26 14:34:21,055 INFO]: Loading KFP component "r_job" from /tmp/tmp.Y35uX5nk4z/ephemeral_component.py (directory "/tmp/tmp.Y35uX5nk4z" and module name "ephemeral_component")
    [KFP Executor 2025-03-26 14:34:21,057 INFO]: Got executor_input:
    {
        "inputs": {
            "parameterValues": {
                "r_libs": [
                    "c",
                    "d"
                ],
                "r_script": "my script here",
                "r_args": [
                    "a",
                    "b"
                ]
            }
        },
        "outputs": {
            "artifacts": {
                "Output": {
                    "artifacts": [
                        {
         

In [114]:
test_r_job.output.metadata

{'encoded_r_script': 'bXkgc2NyaXB0IGhlcmU=',
 'install_command': 'R -e "install.packages(c(\'c\', \'d\'), repos=\'http://cran.us.r-project.org\')" &&',
 'script_args': "'a' 'b'"}

In [117]:
import base64
encoded_bytes = test_r_job.output.metadata['encoded_r_script'].encode('utf-8')
decoded_bytes = base64.b64decode(encoded_bytes)
decoded_r_script = decoded_bytes.decode('utf-8')
decoded_r_script

'my script here'

In [118]:
@kfp.dsl.container_component
def r_container(
    r_job: kfp.dsl.Input[kfp.dsl.Artifact]
):

    return kfp.dsl.ContainerSpec(
        image='r-base:latest',
        command=['sh', '-c'],
        #args = [
        #    f"""
        #        apt-get update && apt-get install -y libcurl4-openssl-dev libssl-dev libxml2-dev && \\
        #        {r_job.metadata['install_command']} \\
        #        Rscript -e '{r_job.metadata['r_script']}' -- {r_job.metadata['script_args']}
        #    """
        #]
        args = [
            """
            apt-get update && apt-get install -y libcurl4-openssl-dev libssl-dev libxml2-dev && 
            {install_command} 
            echo "{encoded_r_script}" | base64 -d > r_script.R &&
            Rscript r_script.R -- {script_args}
            """.format(
                install_command = r_job.metadata['install_command'],
                encoded_r_script = r_job.metadata['encoded_r_script'],
                script_args = r_job.metadata['script_args']
            )
        ]
    )

In [119]:
@kfp.dsl.pipeline(
    name = f'{SERIES}-{EXPERIMENT}',
)
def r_pipeline(
    bq_project: str,
    bq_dataset: str,
    bq_table: str,
    var_target: str,
    var_omit: str,
    r_script: str,
    r_libs: list
) -> kfp.dsl.Artifact:
    
    r_job_op = r_job(
        r_args = [bq_project, bq_dataset, bq_table, var_target, var_omit],
        r_libs = r_libs,
        r_script = r_script
    )
    
    r_run = r_container(
        r_job = r_job_op.output
    )
    return r_job_op.output

In [120]:
test_script = r_script[0:208] + r_script[-76:]
print(test_script)

# library import
library(bigrquery)
library(dplyr)

# inputs
args <- commandArgs(trailingOnly = TRUE)
bq_project <- args[1]
bq_dataset <- args[2]
bq_table <- args[3]
var_target <- args[4]
var_omit <- args[5]

# output
writeLines(paste('project:', bq_project, 'dataset:', bq_dataset))



In [121]:
test_pipeline = r_pipeline(
    bq_project = PROJECT_ID,
    bq_dataset = SERIES,
    bq_table = EXPERIMENT,
    var_target = 'Class',
    var_omit = 'transaction_id',
    #r_script = r_script[0:207],
    r_script = test_script,
    r_libs = ['dplyr', 'bigrquery']
)

14:40:56.673 - INFO - Running pipeline: 'r-pipeline-frameworks'
--------------------------------------------------------------------------------
14:40:56.678 - INFO - Executing task 'r-job'
14:40:56.679 - INFO - Streamed logs:

    Found image 'python:3.11'

    [KFP Executor 2025-03-26 14:41:02,422 INFO]: Looking for component `r_job` in --component_module_path `/tmp/tmp.cNrH8wKoAl/ephemeral_component.py`
    [KFP Executor 2025-03-26 14:41:02,423 INFO]: Loading KFP component "r_job" from /tmp/tmp.cNrH8wKoAl/ephemeral_component.py (directory "/tmp/tmp.cNrH8wKoAl" and module name "ephemeral_component")
    [KFP Executor 2025-03-26 14:41:02,425 INFO]: Got executor_input:
    {
        "inputs": {
            "parameterValues": {
                "pipelinechannel--var_target": "Class",
                "r_args": [
                    "statmike-mlops-349915",
                    "r-pipeline",
                    "frameworks",
                    "Class",
                    "transaction_id"


In [122]:
test_pipeline.output.metadata

{'encoded_r_script': 'IyBsaWJyYXJ5IGltcG9ydApsaWJyYXJ5KGJpZ3JxdWVyeSkKbGlicmFyeShkcGx5cikKCiMgaW5wdXRzCmFyZ3MgPC0gY29tbWFuZEFyZ3ModHJhaWxpbmdPbmx5ID0gVFJVRSkKYnFfcHJvamVjdCA8LSBhcmdzWzFdCmJxX2RhdGFzZXQgPC0gYXJnc1syXQpicV90YWJsZSA8LSBhcmdzWzNdCnZhcl90YXJnZXQgPC0gYXJnc1s0XQp2YXJfb21pdCA8LSBhcmdzWzVdCgojIG91dHB1dAp3cml0ZUxpbmVzKHBhc3RlKCdwcm9qZWN0OicsIGJxX3Byb2plY3QsICdkYXRhc2V0OicsIGJxX2RhdGFzZXQpKQo=',
 'install_command': 'R -e "install.packages(c(\'dplyr\', \'bigrquery\'), repos=\'http://cran.us.r-project.org\')" &&',
 'script_args': "'statmike-mlops-349915' 'r-pipeline' 'frameworks' 'Class' 'transaction_id'"}

In [123]:
import base64
encoded_bytes = test_pipeline.output.metadata['encoded_r_script'].encode('utf-8')
decoded_bytes = base64.b64decode(encoded_bytes)
decoded_r_script = decoded_bytes.decode('utf-8')
decoded_r_script

"# library import\nlibrary(bigrquery)\nlibrary(dplyr)\n\n# inputs\nargs <- commandArgs(trailingOnly = TRUE)\nbq_project <- args[1]\nbq_dataset <- args[2]\nbq_table <- args[3]\nvar_target <- args[4]\nvar_omit <- args[5]\n\n# output\nwriteLines(paste('project:', bq_project, 'dataset:', bq_dataset))\n"

In [124]:
kfp.compiler.Compiler().compile(
    pipeline_func = r_pipeline,
    package_path = f'{DIR}/{SERIES}-{EXPERIMENT}.yaml'
)

In [125]:
pipeline_job = aiplatform.PipelineJob(
    display_name = f"{SERIES}-{EXPERIMENT}",
    template_path = f"{DIR}/{SERIES}-{EXPERIMENT}.yaml",
    parameter_values = dict(
        bq_project = PROJECT_ID,
        bq_dataset = SERIES,
        bq_table = EXPERIMENT,
        var_target = 'Class',
        var_omit = 'transaction_id',
        r_script = test_script,
        r_libs = ['dplyr', 'bigrquery']
    ),
    pipeline_root = f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/pipeline_root',
    enable_caching = None # True (enabled), False (disable), None (defer to component level caching) 
)

In [126]:
response = pipeline_job.submit(
    service_account = SERVICE_ACCOUNT
)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/r-pipeline-frameworks-20250326145001?project=1026793852137


In [127]:
pipeline_job.wait()

PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/r-pipeline-frameworks-20250326145001 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/u

In [128]:
aiplatform.get_pipeline_df(pipeline = f'{SERIES}-{EXPERIMENT}')

,pipeline_name,run_name,param.input:bq_project,param.vertex-ai-pipelines-artifact-argument-binding,param.input:var_target,param.vmlmd_lineage_integration,param.input:bq_dataset,param.input:bq_table,param.input:r_libs,param.input:r_script,param.input:var_omit,metric.encoded_r_script,metric.install_command,metric.script_args,metric.r_script
0,r-pipeline-frameworks,r-pipeline-frameworks-20250326145001,statmike-mlops-349915,{'output:Output': ['projects/1026793852137/loc...,Class,{'pipeline_run_component': {'parent_task_names...,r-pipeline,frameworks,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,transaction_id,IyBsaWJyYXJ5IGltcG9ydApsaWJyYXJ5KGJpZ3JxdWVyeS...,"R -e ""install.packages(c('dplyr', 'bigrquery')...",'statmike-mlops-349915' 'r-pipeline' 'framewor...,NaN
1,r-pipeline-frameworks,r-pipeline-frameworks-20250326121834,statmike-mlops-349915,{'output:Output': ['projects/1026793852137/loc...,Class,{'pipeline_run_component': {'parent_task_names...,r-pipeline,frameworks,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,transaction_id,NaN,"R -e ""install.packages(c('dplyr', 'bigrquery')...",'statmike-mlops-349915' 'r-pipeline' 'framewor...,# library import\nlibrary(bigrquery)\nlibrary(...
2,r-pipeline-frameworks,r-pipeline-frameworks-20250325231842,statmike-mlops-349915,{'output:Output': ['projects/1026793852137/loc...,Class,{'pipeline_run_component': {'parent_task_names...,r-pipeline,frameworks,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,transaction_id,NaN,"R -e ""install.packages(c('dplyr', 'bigrquery')...",'statmike-mlops-349915' 'r-pipeline' 'framewor...,# library import\nlibrary(bigrquery)\nlibrary(...
3,r-pipeline-frameworks,r-pipeline-frameworks-20250325220816,statmike-mlops-349915,{'output:Output': ['projects/1026793852137/loc...,Class,{'pipeline_run_component': {'location_id': 'us...,r-pipeline,frameworks,"[dplyr, bigrquery]",# library import\nlibrary(bigrquery)\nlibrary(...,transaction_id,NaN,"R -e ""install.packages(c('dplyr', 'bigrquery')...",'statmike-mlops-349915' 'r-pipeline' 'framewor...,# library import\nlibrary(bigrquery)\nlibrary(...
